### ------------------------------------------------------------------------------------------------------
## 社会変革型 医療データサイエンティスト育成講座
# Chapter 1: 重回帰モデル
### ------------------------------------------------------------------------------------------------------

#### 1. ライブラリ使用方法

In [ ]:
# データのロード
# データはpandasのDataFrameに入れます
# DataFrameはPythonにおける基本中の基本ですが、オプションが多く使いこなすのはなかなか難しいです

# まずは"import"によってpandasライブラリを使用できる状態にします
# "as pd"の部分は必ずしもpdではなくてもOKですが、pdという命名がデファクトスタンダードになっています
import pandas as pd

# pandasの中に入っている関数であるread_csvを使用して、データをロードします
# ホームディレクトリにデータを格納していれば、フォルダの指定は必要ありません
bace_data = pd.read_csv('~/bace_data.csv')

# DataFrameの中身をインスペクトするために、しばしば"head()"を用います
bace_data.head()

In [ ]:
# 説明変数として分子量 (MW)およびAlogP, 目的変数としてpIC50を使用します
# こちらも頻用される数値計算用ライブラリであるnumpyのアレイに変換しておきます
import numpy as np
X = np.array(bace_data[['MW','AlogP']])
y = np.array(bace_data['pIC50'])

In [ ]:
# X（説明変数）とy（目的変数）に何が入っているか確認します
print('X: ', X)
print('y: ', y)

In [ ]:
# 可視化のためのライブラリであるmatplotlibを用いてデータを可視化します
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

fig = plt.figure(figsize=[12,9]) 
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:,0],X[:,1],y)
ax.set_xlabel('MW')
ax.set_ylabel('AlogP')
ax.set_zlabel('pIC50')
plt.show()

In [ ]:
# sklearn.linear_modelによる線形回帰モデル構築


# インスタンスの作成


# データのフィッティング


# 結果の出力
print('偏回帰係数 (MW): ', )
print('偏回帰係数 (AlogP): ', )
print('切片: ', )

In [ ]:
# ライブラリの詳細について調べたい場合には、ウェブページを見るか以下のように?でヘルプを表示します
?LinearRegression

In [ ]:
# (参考) 回帰の結果の可視化
x1 = np.linspace(0,1400,100)
x2 = np.linspace(-4,9,100)
X1,X2 = np.meshgrid(x1,x2)
yy = np.zeros([100,100])
for i,xx1 in enumerate(x1):
    for j,xx2 in enumerate(x2):
        yy[i,j] = model.intercept_ + xx1*model.coef_[0] 
        + xx2*model.coef_[1]
        
fig = plt.figure(figsize=[8,6]) 
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X1, X2, yy, color='r', alpha=0.3)
ax.scatter(X[:,0],X[:,1],y)
ax.set_xlabel('MW')
ax.set_ylabel('AlogP')
ax.set_zlabel('pIC50')
plt.show()

In [ ]:
# (参考) statsmodelsによる線形回帰モデル構築
import statsmodels.api as sm

# "add_constant"を行う必要があります
X = sm.add_constant(X)

# OLSはordinary least squareのことです
mod = sm.OLS(y, X)
res = mod.fit()

# summary機能があるのでとても便利！
print(res.summary())

#### 2. データ標準化

In [ ]:
# 説明変数と目的変数の定義
X = np.array(bace_data[['MW','AlogP']])
y = np.array(bace_data[['pIC50']]) # 2次元データとする必要がある

In [ ]:
# StandardScalerを使用したデータの標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_st = scaler.fit_transform(X)
y_st = scaler.fit_transform(y)

In [ ]:
# (参考) ヒストグラムを用いた標準化の確認
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(1,2)
ax[0].hist(X[:,0])
ax[0].set_xlabel('before st')
ax[1].hist(X_st[:,0])
ax[1].set_xlabel('after st')
plt.show()

In [ ]:
# (参考) 標準化なしのデータで線形モデル構築
import statsmodels.api as sm
X_c = sm.add_constant(X)
mod = sm.OLS(y,X_c)
res = mod.fit()
print(res.summary())

In [ ]:
# (参考) 標準化したデータでモデル構築
X_st_c = sm.add_constant(X_st)
mod = sm.OLS(y_st,X_st_c)
res = mod.fit()
print(res.summary())

#### (おまけ) 多重共線性

In [ ]:
# サンプル生成関数の定義

def generate_sample_2d(params,nsample=50,xrange=[[0,10],[0,10]],collinear=False):
    rsample = np.zeros([nsample,3])
    for i in range(nsample):
        err = np.random.normal(0,params['sigma'],1)
        if collinear:
            err_collin = np.random.normal(0,params['sigma_collin'],1)
            x_1 = np.random.uniform(xrange[0][0],xrange[0][1],1)
            x_2 = x_1*params['b_1_collin']+err_collin          
        else:
            x_1 = np.random.uniform(xrange[0][0],xrange[0][1],1)
            x_2 = np.random.uniform(xrange[1][0],xrange[1][1],1)
        y = params['b_0'] + params['b_1']*x_1 + params['b_2']*x_2 + err
        rsample[i,:] = y[0],x_1,x_2
        
    return rsample

In [ ]:
# 試しにプロット

from mpl_toolkits.mplot3d import Axes3D

params = {'b_0': 3,
          'b_1': 2.5,
          'b_2': 1.2,
          'sigma': 10
          }

rsample = generate_sample_2d(params)
fig = plt.figure() 
ax = fig.add_subplot(111, projection='3d')
ax.scatter(rsample[:,1],rsample[:,2],rsample[:,0])
plt.show()

In [ ]:
# statsmodelsを使用してモデルをフィッティングします

params = {'b_0': 1,
          'b_1': 1,
          'b_2': 1,
          'sigma': 5
          }
rsample = generate_sample_2d(params,nsample=100)

# 定数に相当する列をデータに付加
x = sm.add_constant(rsample[:,1:])

mod = sm.OLS(rsample[:,0], x)
res = mod.fit()

print(res.summary())

In [ ]:
# 多重共線性ありの場合のサンプル生成

params = {'b_0': 1,
          'b_1': 1,
          'b_2': 1,
          'sigma': 5,
          'sigma_collin': 2,
          'b_1_collin': 1
          }

fig, ax = plt.subplots(1,2,figsize=[8,4],sharex=True,sharey=True)
rsample_1 = generate_sample_2d(params)
rsample_2 = generate_sample_2d(params,collinear=True)

ax[0].scatter(rsample_1[:,1],rsample_1[:,2])
ax[0].set_xlabel('x_1')
ax[0].set_ylabel('x_2')
ax[0].set_title('without collinearity')
ax[1].scatter(rsample_2[:,1],rsample_2[:,2])
ax[1].set_title('with collinearity')
plt.show()

In [ ]:
# 3d plot

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=[10,4]) 
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')
ax1.scatter(rsample_1[:,1],rsample_1[:,2],rsample_1[:,0])
ax1.set_title('without collinearity')
ax2.scatter(rsample_2[:,1],rsample_2[:,2],rsample_2[:,0])
ax2.set_title('with collinearity')
plt.show()

In [ ]:
# 多重共線性が認められるデータに対するモデル当てはめ

params = {'b_0': 1,
          'b_1': 1,
          'b_2': 1,
          'sigma': 5,
          'sigma_collin': 0.5,
          'b_1_collin': 1
          }

rsample = generate_sample_2d(params,nsample=100,collinear=True)
x = sm.add_constant(rsample[:,1:])
mod = sm.OLS(rsample[:,0], x)
res = mod.fit()

print(res.summary())

In [ ]:
params = {'b_0': 3,
          'b_1': 2.5,
          'b_2': 1.2,
          'sigma': 7,
          'sigma_collin': 1,
          'b_1_collin': 1
          }

# sigmaを0.1ずつ変化させる
sigma_ = np.linspace(0.1,5,50)

# 空の配列を作成しておく
r_2 = np.zeros([len(sigma_),1]) 
b_2_se = np.zeros([len(sigma_),1])

# sigmaの数だけループを回す
for i,s in enumerate(sigma_):
    
    # sigmaをループごとに変化させていく
    params['sigma_collin'] = s
    
    # モデルフィッティング
    rsample = generate_sample_2d(params,nsample=100,collinear=True)
    x = sm.add_constant(rsample[:,1:])
    mod = sm.OLS(rsample[:,0], x)
    res = mod.fit()
    
    # 決定係数と回帰係数
    r_2[i] = res.rsquared
    b_2_se[i] = res.bse[1]
    
# プロット
plt.plot(sigma_,b_2_se)
plt.axhline(y=0.2, color='r', linestyle='--')
plt.xlabel('sigma')
plt.ylabel('SE of b1')
plt.legend(['with collinearity','without collinearity'])
plt.show()

plt.plot(sigma_,r_2)
plt.xlabel('sigma')
plt.ylabel('r-squared')
plt.show()